In [ ]:
#!/usr/bin/env python

## Optional step: Remove warnings thrown by invalid SSL certificate.
import warnings
warnings.filterwarnings('ignore')

## 
import os, sys
import pandas as pd

In [ ]:
def eprint(log_str, log_file='./mylog.txt', newline=f"\n"):
    with open(log_file, 'a') as ifh:
        ifh.write(log_str + newline)
        print(log_str)
    return None

def timekeeper(timepoint='start', rootdir=".", label='timekeeper'):
    ## get folder & log file
    assert timepoint.lower() in ['start', 'end'], f'<f{timepoint}> is not in ["start", "end"]'
    datetime_file = f"{rootdir}/{label}.{timepoint.lower()}date"

    ## get datetime
    import datetime
    datetime_str = datetime.datetime.now()
    
    ## define command
    cmd = f"echo {datetime_str} > {datetime_file}; chmod 775 {datetime_file}"
    os.system(cmd)

    ## export
    print(f"\tcmd = {cmd}")
    return f"cmd = {cmd}"

In [1]:
cwd = os.getcwd()
tpdnav_rootdir = f"/fsx/data/AUTOMATION/TPDNav"

tpdnav_Kymera_D360 = f"{tpdnav_rootdir}/KymeraMols.tpdecomp.D360.csv"

## ---------------------------------------------------------
timekeeper('start', rootdir=tpdnav_rootdir, label='tpdnav_d360push')

## ---------------------------------------------------------
eprint("Reading data from {}".format(tpdnav_Kymera_D360))
tpdf = pd.read_csv(tpdnav_Kymera_D360)
print("tpdf.head():")
print(tpdf.head())
eprint("len(tpdf.index) = {}".format(len(tpdf.index)))

## ---------------------------------------------------------







## ---------------------------------------------------------
timekeeper('end', rootdir=tpdnav_rootdir, label='tpdnav_d360push')

## ---------------------------------------------------------
eprint("Done.")
sys.stdout.flush()
sys.stderr.flush()


## ---------------------------------------------------------


In [ ]:
db_access = {"host": "10.3.30.118", "port": "5432", "user": "cadd_rw", "password": "k5$ypghw$X!RRC^)"}

def connect_2_postgre(database, db_access):
    # server, port, dbname = "10.3.30.118", "5432", "compound_decomposition"
    # uname, psswd = "cadd_rw", "k5$ypghw$X!RRC^)"

   host=db_access["host"], port=db_access["port"], user=db_access["user"], password=db_access["password"]"k5$ypghw$X!RRC^)"

   try:
      import psycopg2
      
      ## ---------- build connection ----------
      print(f"\ts1. Trying to connect ...")
      conn = psycopg2.connect(user=user, password=password, host=host, port=port, database=database)
      cursor = conn.cursor()

      ## blow away all rows of data in the kymera_protac_decomposition table
      dquery = """delete from prod.kymera_protac_decomposition"""
      print(f"dquery = {dquery}")
      cursor.execute(dquery)
      conn.commit()
      
      ## 
      print("Just blew away all rows of data in kymera_protac_decomposition table ...\n")
      print(f"Bulk uploading the data in file {tpdnav_Kymera_D360} to the D360 postgres DB ...")

      sqlcmd = "COPY prod.kymera_protac_decomposition from STDIN DELIMITER ',' CSV HEADER;"
      eprint("sqlcmd = {}".format(sqlcmd))
      #cursor.copy_expert(sqlcmd, open("/fsx/data/AUTOMATION/TPDNav/KymeraMols.tpdecomp.D360.csv", "r"))
      cursor.copy_expert(sqlcmd, open(tpdnav_Kymera_D360, "r"))
      conn.commit()
      conn.close()

   except Exception as error:
      eprint("Error while communicating to PostgreSQL", error)

In [ ]:


import sys, os, getopt
import csv
import pandas as pd

from PatGlobalVars import *

d360columns = ["PROTAC_id","Decomp_Flag","Decomp_Level","E3_smi","E3_smi_stub","Linker_smi","Linker_smi_stub",
"POI_smi","POI_smi_stub","Lib_E3_target","Lib_POI_target","KymE3L_Chemotype","KymPOI_Chemotype","E3_flag","POI_flag"]





import psycopg2
from psycopg2 import Error

def main(argv):
   global outdir,outtsv,qid
   updatefreq = 1000

   ##
   eprint("Reading data from {}".format(tpdnav_Kymera_D360))
   tpdf = pd.read_csv(tpdnav_Kymera_D360)
   print("tpdf.head():")
   print(tpdf.head())
   eprint("len(tpdf.index) = {}".format(len(tpdf.index)))

   try:
      eprint("Trying to connect ...")
      conn = psycopg2.connect(user="cadd_rw", password="k5$ypghw$X!RRC^)", host="10.3.30.118", port="5432", database="compound_decomposition")
      cursor = conn.cursor()

      # blow away all rows of data in the kymera_protac_decomposition table
      dquery = """delete from prod.kymera_protac_decomposition"""
      eprint("dquery = {}".format(dquery))
      cursor.execute(dquery)
      conn.commit()
      eprint("Just blew away all rows of data in kymera_protac_decomposition table ...\n")

      eprint("Bulk uploading the data in file {} to the D360 postgres DB ...".format(tpdnav_Kymera_D360))

      sqlcmd = "COPY prod.kymera_protac_decomposition from STDIN DELIMITER ',' CSV HEADER;"
      eprint("sqlcmd = {}".format(sqlcmd))
      #cursor.copy_expert(sqlcmd, open("/fsx/data/AUTOMATION/TPDNav/KymeraMols.tpdecomp.D360.csv", "r"))
      cursor.copy_expert(sqlcmd, open(tpdnav_Kymera_D360, "r"))
      conn.commit()
      conn.close()

   except (Exception, Error) as error:
      eprint("Error while connecting to PostgreSQL",error)

   ## -------------------------------------

# ===========================================================================================
if(__name__ == "__main__"):
   main(sys.argv[1:])
